In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from pylab import rcParams
rcParams['figure.figsize'] = 12,8

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns


import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import re

import os
from glob import glob
from tqdm import tqdm

import yaml
from yaml import dump
import uuid
import itertools

import matplotlib.pyplot as plt

In [3]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [4]:
import warnings
warnings.simplefilter(action="ignore")

## classification

In [36]:
path_c = glob(f"/diploma_info/datalake/statistical_result/classification/*.csv")
path_c

['/diploma_info/datalake/statistical_result/classification\\statistical_result_amazon_daily_20240330.csv',
 '/diploma_info/datalake/statistical_result/classification\\statistical_result_amazon_daily_20240415.csv',
 '/diploma_info/datalake/statistical_result/classification\\statistical_result_amazon_daily_20240416.csv',
 '/diploma_info/datalake/statistical_result/classification\\statistical_result_amazon_daily_20240416_1.csv',
 '/diploma_info/datalake/statistical_result/classification\\statistical_result_apple_daily_20240330.csv',
 '/diploma_info/datalake/statistical_result/classification\\statistical_result_apple_daily_20240415.csv',
 '/diploma_info/datalake/statistical_result/classification\\statistical_result_apple_daily_20240416.csv',
 '/diploma_info/datalake/statistical_result/classification\\statistical_result_apple_daily_20240416_1.csv',
 '/diploma_info/datalake/statistical_result/classification\\statistical_result_google_daily_20240330.csv',
 '/diploma_info/datalake/statistical_

In [30]:
amazon_c = pd.DataFrame()
apple_c = pd.DataFrame()
google_c = pd.DataFrame()
meta_c = pd.DataFrame()
netflix_c = pd.DataFrame()


for company in ['amazon', 'apple', 'google', 'meta', 'netflix']:
    for p in path_c:
        if company in p:
            amazon_c = pd.concat([amazon_c, pd.read_csv(p)])
            apple_c = pd.concat([apple_c, pd.read_csv(p)])
            google_c = pd.concat([google_c, pd.read_csv(p)])
            meta_c = pd.concat([meta_c, pd.read_csv(p)])
            netflix_c = pd.concat([netflix_c, pd.read_csv(p)])

In [31]:
companies_c = {'amazon':amazon_c, 'apple':apple_c, 'google':google_c, 'meta':meta_c, 'netflix':netflix_c}
forecasted_days = 4
models_c = ['xgboost', 'lightgbm', 'random_forest', 'knear_neighbors', 'logistic_regression']

col = 'accuracy'

In [32]:
best_results_c_dict = {}

for i, (company, day, model) in enumerate(itertools.product(list(companies_c.items()), range(forecasted_days), models_c)):
#     print(i, _, day, model)

    company_name = company[0]
    _ = company[1]
    
    temp_df = _[_['model'] == model][_[col] == _[(_['forecasted_day'] == day)&(_['model'] == model)][col].max()]
    # [['features', 'hyperparameters']].values
    
    temp_df = temp_df.rename(columns={"Unnamed: 0": "exp_name"})
    
    d = {'company':company_name}
    d.update(dict(zip(temp_df.columns, temp_df.values[0])))
    
    best_results_c_dict[f'{i}'] = d
    
    
best_results_c = pd.DataFrame(best_results_c_dict).T

In [33]:
best_results_c[(best_results_c['company'] == 'amazon') & (best_results_c['forecasted_day'] == 3)]

,company,exp_name,forecasted_day,n_days_forecasted,n_forecasted_right,accuracy,model,train_start/duration,features,hyperparameters,experiment_uuid
3,amazon,knear_neighbors_9151_d-3,3.0,24.0,16.0,66.67,knear_neighbors,2015-01-01,"['close_lag_1', 'volume_lag_1']","{'n_jobs': -1, 'n_neighbors': 50}",af49a062-fbca-11ee-9151-c0e434d84b22
15,amazon,xgboost_bb3d_d-3,3.0,24.0,15.0,62.5,xgboost,2015-01-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'booster': 'gbtree', 'colsample_bytree': 1, '...",af49a05d-fbca-11ee-bb3d-c0e434d84b22
16,amazon,lightgbm_b091_d-3,3.0,24.0,16.0,66.67,lightgbm,2015-01-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'eta': 0.3, 'max_depth': 7, 'metric': 'binary...",af49a05f-fbca-11ee-b091-c0e434d84b22
17,amazon,random_forest_9e1f_d-3,3.0,24.0,15.0,62.5,random_forest,2010-01-01,"['close_lag_1', 'volume_lag_1']","{'max_depth': 7, 'n_estimators': 5000, 'n_jobs...",af49a058-fbca-11ee-9e1f-c0e434d84b22
18,amazon,knear_neighbors_9151_d-3,3.0,24.0,16.0,66.67,knear_neighbors,2015-01-01,"['close_lag_1', 'volume_lag_1']","{'n_jobs': -1, 'n_neighbors': 50}",af49a062-fbca-11ee-9151-c0e434d84b22
19,amazon,logistic_regression_b276_d-3,3.0,24.0,16.0,66.67,logistic_regression,1997-05-15,"['close_lag_1', 'close_max_3_days', 'close_min...","{'max_iter': 10, 'n_jobs': -1, 'random_state':...",6a500347-fb2b-11ee-b276-c0e434d84b22


In [34]:
best_best_results_c_dict = {}

for i, (company, day) in enumerate(itertools.product(list(companies_c.keys()), range(forecasted_days))):
    
    _ = best_results_c.copy()
    
    temp_df = _[_['company'] == company][_[col] == _[(_['forecasted_day'] == day)&(_['company'] == company)][col].max()]
    # [['features', 'hyperparameters']].values
    
    
    best_best_results_c_dict[f'{i}'] = dict(zip(temp_df.columns, temp_df.values[0]))
    
    
best_best_results_c = pd.DataFrame(best_best_results_c_dict).T

In [35]:
best_best_results_c

,company,exp_name,forecasted_day,n_days_forecasted,n_forecasted_right,accuracy,model,train_start/duration,features,hyperparameters,experiment_uuid
0,amazon,random_forest_836d_d-0,0.0,27.0,17.0,62.96,random_forest,2010-01-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'max_depth': 7, 'n_estimators': 5000, 'n_jobs...",af49a059-fbca-11ee-836d-c0e434d84b22
1,amazon,lightgbm_b091_d-1,1.0,26.0,19.0,73.08,lightgbm,2015-01-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'eta': 0.3, 'max_depth': 7, 'metric': 'binary...",af49a05f-fbca-11ee-b091-c0e434d84b22
2,amazon,xgboost_a077_d-2,2.0,25.0,16.0,64.0,xgboost,2010-01-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'booster': 'gbtree', 'colsample_bytree': 1, '...",af49a055-fbca-11ee-a077-c0e434d84b22
3,amazon,knear_neighbors_9151_d-3,3.0,24.0,16.0,66.67,knear_neighbors,2015-01-01,"['close_lag_1', 'volume_lag_1']","{'n_jobs': -1, 'n_neighbors': 50}",af49a062-fbca-11ee-9151-c0e434d84b22
4,apple,xgboost_84e2_d-0,0.0,27.0,19.0,70.37,xgboost,2015-01-01,"['close_lag_1', 'volume_lag_1']","{'booster': 'gbtree', 'colsample_bytree': 1, '...",1a44aade-fbd0-11ee-84e2-c0e434d84b22
5,apple,xgboost_af2c_d-1,1.0,26.0,17.0,65.38,xgboost,2010-01-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'booster': 'gbtree', 'colsample_bytree': 1, '...",1a44aac9-fbd0-11ee-af2c-c0e434d84b22
6,apple,xgboost_af2c_d-2,2.0,25.0,17.0,68.0,xgboost,2010-01-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'booster': 'gbtree', 'colsample_bytree': 1, '...",1a44aac9-fbd0-11ee-af2c-c0e434d84b22
7,apple,xgboost_af2c_d-3,3.0,24.0,15.0,62.5,xgboost,2010-01-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'booster': 'gbtree', 'colsample_bytree': 1, '...",1a44aac9-fbd0-11ee-af2c-c0e434d84b22
8,google,lightgbm_a89c_d-0,0.0,27.0,19.0,70.37,lightgbm,2010-01-01,"['close_lag_1', 'volume_lag_1']","{'eta': 0.3, 'max_depth': 7, 'metric': 'binary...",0e20a838-fbd3-11ee-a89c-c0e434d84b22
9,google,logistic_regression_8584_d-1,1.0,26.0,18.0,69.23,logistic_regression,2004-08-19,"['close_lag_1', 'volume_lag_1']","{'max_iter': 10, 'n_jobs': -1, 'random_state':...",6cccdb14-fb2b-11ee-8584-c0e434d84b22


## regression

In [12]:
path_r = glob(f"/diploma_info/datalake/statistical_result/regression/*.csv")

In [13]:
amazon_r = pd.read_csv(path_r[0])
apple_r = pd.read_csv(path_r[1])
google_r = pd.read_csv(path_r[2])
meta_r = pd.read_csv(path_r[3])
netflix_r = pd.read_csv(path_r[4])

In [14]:
companies_r = {'amazon':amazon_r, 'apple':apple_r, 'google':google_r, 'meta':meta_r, 'netflix':netflix_r}
forecasted_days = 4
models_r = ['xgboost', 'lightgbm', 'random_forest', 'linear_regression']

col = 'accuracy'

In [15]:
amazon_r

,Unnamed: 0,forecasted_day,n_days_forecasted,n_days_err_less_5,%_days_err_less_5,accuracy,mean_abs_err,mean_rel_err,median_abs_err,median_rel_err,...,q25_rel_err,q75_abs_err,q75_rel_err,max_abs_err,max_rel_err,model,train_start/duration,features,hyperparameters,experiment_uuid
0,xgboost_8e36_d-0,0.0,27.0,24.0,88.89,97.3641,4.0304,2.6359,3.57,2.270,...,1.2250,5.2000,3.3350,16.00,9.31,xgboost,2004-09-01,"['close_lag_1', 'volume_lag_1']","{'booster': 'gbtree', 'colsample_bytree': 1, '...",7ee5c858-ec75-11ee-8e36-c0e434d84b22
1,xgboost_8e36_d-1,1.0,26.0,20.0,76.92,96.6573,4.9273,3.3427,3.86,2.370,...,1.1925,5.6150,3.5850,20.67,12.03,xgboost,2004-09-01,"['close_lag_1', 'volume_lag_1']","{'booster': 'gbtree', 'colsample_bytree': 1, '...",7ee5c858-ec75-11ee-8e36-c0e434d84b22
2,xgboost_8e36_d-2,2.0,25.0,18.0,72.00,95.9992,5.8332,4.0008,4.71,3.390,...,1.5400,8.9300,5.6200,20.24,11.88,xgboost,2004-09-01,"['close_lag_1', 'volume_lag_1']","{'booster': 'gbtree', 'colsample_bytree': 1, '...",7ee5c858-ec75-11ee-8e36-c0e434d84b22
3,xgboost_8e36_d-3,3.0,24.0,15.0,62.50,95.3375,7.1000,4.6625,5.54,3.565,...,2.3450,8.9325,5.6900,20.46,12.00,xgboost,2004-09-01,"['close_lag_1', 'volume_lag_1']","{'booster': 'gbtree', 'colsample_bytree': 1, '...",7ee5c858-ec75-11ee-8e36-c0e434d84b22
4,xgboost_8e36_d-4,4.0,23.0,15.0,65.22,94.7539,8.5026,5.2461,5.95,3.810,...,3.0950,12.0300,7.4850,20.46,12.00,xgboost,2004-09-01,"['close_lag_1', 'volume_lag_1']","{'booster': 'gbtree', 'colsample_bytree': 1, '...",7ee5c858-ec75-11ee-8e36-c0e434d84b22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,linear_regression_8eaa_d-0,0.0,27.0,26.0,96.30,98.6052,0.6059,1.3948,0.79,1.010,...,0.7400,1.8000,1.4550,12.52,7.29,linear_regression,2004-09-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'fit_intercept': False, 'n_jobs': -1, 'positi...",7ee5c899-ec75-11ee-8eaa-c0e434d84b22
326,linear_regression_8eaa_d-1,1.0,26.0,24.0,92.31,97.6296,1.5550,2.3704,0.63,1.790,...,0.4350,3.2850,2.9650,16.82,9.87,linear_regression,2004-09-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'fit_intercept': False, 'n_jobs': -1, 'positi...",7ee5c899-ec75-11ee-8eaa-c0e434d84b22
327,linear_regression_8eaa_d-2,2.0,25.0,20.0,80.00,97.0132,2.4096,2.9868,1.47,1.950,...,1.1400,5.5400,3.9000,16.82,9.87,linear_regression,2004-09-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'fit_intercept': False, 'n_jobs': -1, 'positi...",7ee5c899-ec75-11ee-8eaa-c0e434d84b22
328,linear_regression_8eaa_d-3,3.0,24.0,19.0,79.17,97.0992,3.6262,2.9008,2.47,1.875,...,1.0425,5.5175,3.8325,17.09,10.02,linear_regression,2004-09-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'fit_intercept': False, 'n_jobs': -1, 'positi...",7ee5c899-ec75-11ee-8eaa-c0e434d84b22


In [16]:
best_results_r_dict = {}

for i, (company, day, model) in enumerate(itertools.product(list(companies_r.items()), range(forecasted_days), models_r)):
#     print(i, company[0], day, model)

    company_name = company[0]
    _ = company[1]
    
    temp_df = _[_['model'] == model][_[col] == _[(_['forecasted_day'] == day)&(_['model'] == model)][col].max()]
    # [['features', 'hyperparameters']].values
    temp_df = temp_df.rename(columns={"Unnamed: 0": "exp_name"})
    
    d = {'company':company_name}
    d.update(dict(zip(temp_df.columns, temp_df.values[0])))
    
    best_results_r_dict[f'{i}'] = d
    
    
best_results_r = pd.DataFrame(best_results_r_dict).T

In [17]:
best_results_r[(best_results_r['company'] == 'netflix') & (best_results_r['forecasted_day'] == 0)]

,company,exp_name,forecasted_day,n_days_forecasted,n_days_err_less_5,%_days_err_less_5,accuracy,mean_abs_err,mean_rel_err,median_abs_err,...,q25_rel_err,q75_abs_err,q75_rel_err,max_abs_err,max_rel_err,model,train_start/duration,features,hyperparameters,experiment_uuid
64,netflix,xgboost_8544_d-0,0.0,27.0,25.0,92.59,97.9063,-0.7567,2.0937,-1.9,...,0.62,3.19,2.47,54.78,10.05,xgboost,2004-09-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'booster': 'gbtree', 'colsample_bytree': 1, '...",a7c7b980-ecec-11ee-8544-c0e434d84b22
65,netflix,lightgbm_a483_d-0,0.0,27.0,0.0,0.0,0.1952,516.5193,99.8048,491.19,...,99.79,561.03,99.82,574.79,99.83,lightgbm,2004-09-01,"['close_lag_1', 'volume_lag_1']","{'eta': 0.3, 'max_depth': 7, 'metric': 'binary...",a7c7b985-ecec-11ee-a483-c0e434d84b22
66,netflix,random_forest_85a3_d-0,0.0,27.0,26.0,96.3,98.137,3.3822,1.863,1.75,...,0.615,9.04,2.035,53.45,9.81,random_forest,2004-09-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'max_depth': 7, 'n_estimators': 5000, 'n_jobs...",a7c7b99e-ecec-11ee-85a3-c0e434d84b22
67,netflix,linear_regression_9784_d-0,0.0,27.0,26.0,96.3,98.5659,2.7019,1.4341,1.34,...,0.44,5.145,1.87,52.63,9.66,linear_regression,2004-09-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'fit_intercept': True, 'n_jobs': -1, 'positiv...",a7c7b9ae-ecec-11ee-9784-c0e434d84b22


In [18]:
best_best_results_r_dict = {}

for i, (company, day) in enumerate(itertools.product(list(companies_r.keys()), range(forecasted_days))):
    
    _ = best_results_r.copy()
    
    temp_df = _[_['company'] == company][_[col] == _[(_['forecasted_day'] == day)&(_['company'] == company)][col].max()]
    # [['features', 'hyperparameters']].values
    
    
    best_best_results_r_dict[f'{i}'] = dict(zip(temp_df.columns, temp_df.values[0]))
    
    
best_best_results_r = pd.DataFrame(best_best_results_r_dict).T

In [19]:
best_best_results_r

,company,exp_name,forecasted_day,n_days_forecasted,n_days_err_less_5,%_days_err_less_5,accuracy,mean_abs_err,mean_rel_err,median_abs_err,...,q25_rel_err,q75_abs_err,q75_rel_err,max_abs_err,max_rel_err,model,train_start/duration,features,hyperparameters,experiment_uuid
0,amazon,random_forest_a9b6_d-0,0.0,27.0,26.0,96.3,98.663,0.3307,1.337,-0.44,...,0.615,1.55,1.385,12.56,7.31,random_forest,2004-09-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'max_depth': 9, 'n_estimators': 5000, 'n_jobs...",7ee5c88c-ec75-11ee-a9b6-c0e434d84b22
1,amazon,random_forest_a30f_d-1,1.0,26.0,24.0,92.31,97.6588,1.1085,2.3412,0.5,...,0.42,2.1175,3.4225,16.18,9.5,random_forest,2004-09-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'max_depth': 7, 'n_estimators': 10, 'n_jobs':...",7ee5c87d-ec75-11ee-a30f-c0e434d84b22
2,amazon,random_forest_a30f_d-2,2.0,25.0,20.0,80.0,97.136,1.858,2.864,1.17,...,0.87,3.61,3.44,16.18,9.5,random_forest,2004-09-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'max_depth': 7, 'n_estimators': 10, 'n_jobs':...",7ee5c87d-ec75-11ee-a30f-c0e434d84b22
3,amazon,random_forest_a30f_d-3,3.0,24.0,18.0,75.0,97.2925,3.0058,2.7075,1.72,...,0.675,4.2175,3.6525,16.4,9.62,random_forest,2004-09-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'max_depth': 7, 'n_estimators': 10, 'n_jobs':...",7ee5c87d-ec75-11ee-a30f-c0e434d84b22
4,apple,linear_regression_9ba3_d-0,0.0,27.0,27.0,100.0,98.9537,-0.2207,1.0463,-0.64,...,0.42,1.36,1.3,5.91,3.76,linear_regression,2004-09-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'fit_intercept': True, 'n_jobs': -1, 'positiv...",62f05269-ec7d-11ee-9ba3-c0e434d84b22
5,apple,random_forest_a694_d-1,1.0,26.0,24.0,92.31,98.2242,-0.7181,1.7758,0.06,...,0.3325,1.55,2.715,10.5,5.58,random_forest,2004-09-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'max_depth': 7, 'n_estimators': 10, 'n_jobs':...",62f02b8b-ec7d-11ee-a694-c0e434d84b22
6,apple,xgboost_98a6_d-2,2.0,25.0,24.0,96.0,97.7328,-1.8676,2.2672,-1.5,...,0.82,1.54,3.61,7.29,5.39,xgboost,2004-09-01,"['close_lag_1', 'volume_lag_1']","{'booster': 'gbtree', 'colsample_bytree': 1, '...",62f00451-ec7d-11ee-98a6-c0e434d84b22
7,apple,xgboost_a6dc_d-3,3.0,24.0,22.0,91.67,97.6158,2.3642,2.3842,2.455,...,0.975,5.3025,3.115,11.39,5.88,xgboost,2004-09-01,"['close_lag_1', 'close_max_3_days', 'close_min...","{'booster': 'gbtree', 'colsample_bytree': 1, '...",62f0044f-ec7d-11ee-a6dc-c0e434d84b22
8,google,linear_regression_90f8_d-0,0.0,27.0,26.0,96.3,98.8737,0.1804,1.1263,0.72,...,0.43,1.26,1.275,3.26,7.94,linear_regression,2004-09-01,"['close_lag_1', 'year', 'month', 'day', 'day_o...","{'fit_intercept': False, 'n_jobs': -1, 'positi...",9f1633a6-ecdc-11ee-90f8-c0e434d84b22
9,google,xgboost_b3ac_d-1,1.0,26.0,25.0,96.15,97.8327,2.1412,2.1673,2.1,...,1.2475,3.605,2.6975,13.91,8.98,xgboost,2004-09-01,"['close_lag_1', 'volume_lag_1']","{'booster': 'gbtree', 'colsample_bytree': 1, '...",9f163374-ecdc-11ee-b3ac-c0e434d84b22
